In [ ]:
from gurobipy import *

number_job = 12  # number_job, including the source and sink node
processing_time = [0,3,5,1,3,2,4,5,6,4,4,0]
resource_consumption = [4,2,3,3,1,1,2,3,1,1,1,4]
Job_successors = [[1,2,3],[4],[5,7],[5],[6],[7,8,9],[10],[10],[11],[11],[11],[]]
Forbidden_Set = [[1,2],[1,3],[1,7],[2,3],[2,6],[3,6],[4,7,8],[4,7,9],[4,8,9],[6,7],[7,8,9]]


# Initial the gurobi model
m = Model()

#add variables 
S = m.addVars(number_job,vtype=GRB.INTEGER, name='Start_time')
z = m.addVars(number_job,number_job,vtype=GRB.BINARY, name='z')


# Set the obj which means the minmize the makespan
m.setObjective(S[number_job-1],GRB.MINIMIZE)

# add constraints
for i in range(number_job-1):
    for j in range(i+1, number_job):
        m.addConstr(z[i,j] + z[j,i] <= 1)

for i in range(number_job):
    for j in range(number_job):
        for h in range(number_job):
            if((i !=j) and (i != h) and (j != h)):
                m.addConstr(z[i,j] + z[j,h] - z[i,h] <= 1)


for i in range(number_job):
    if(len(Job_successors[i]) != 0):
        for j in Job_successors[i]:
            m.addConstr(z[i,j] == 1)



for i in range(number_job):
    for j in range(number_job):
        if(i != j):
            m.addConstr(S[j] - S[i] + (1-z[i,j])*1000 >= processing_time[i])  # here 1000 means big M in the MIP.

            
for k in range(len(Forbidden_Set)):
    m.addConstr(quicksum(z[i,j] for i in (Forbidden_Set[k]) for j in (Forbidden_Set[k]) if i != j) >= 1)
        

#    m.write('lnear_model11.lp')
m.optimize()

#    Get the solution
output_dict={}
vars =  m.getVars()
for i in range(number_job):
    print('%s %g' % (vars[i].varName, vars[i].x))
    output_dict[i]=vars[i].x

start=list(output_dict.values())
finish=[]
count=0
for iteration in output_dict.values():
    finish.append(iteration+processing_time[count])
    count+=1

# print('Obj: %g' % m.objVal)
print('start',start)
print("finish",finish)

In [ ]:
from datetime import datetime  
from datetime import timedelta

import plotly.express as px
import pandas as pd

# organize the data (absolute date time) to plotly
# suppose we use 2020-01-01 as the baseline time
start_date=[]
finish_date =[]

start_date_base = datetime.strptime('2020-01-01','%Y-%m-%d')

for i in range(number_job):
    start_date.append((start_date_base + timedelta(days=start[i])).strftime('%Y-%m-%d'))
    finish_date.append((start_date_base + timedelta(days=finish[i])).strftime('%Y-%m-%d'))

row_list = []
for i in range(1, number_job-1):
    dict1 = dict(Task = "Job"+str(i), Start = start_date[i], Finish = finish_date[i])
    row_list.append(dict1)


df = pd.DataFrame(row_list)

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Task")
# fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()